<a href="https://colab.research.google.com/github/stefymojica/sample-rag-llm/blob/main/rag_llm_productos_tecnologia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modelo RAG - LLM con productos de tecnologia**

In [44]:
pip install sentence-transformers pandas scikit-learn

In [46]:
pip install torch transformers

In [56]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM

In [102]:
# class SimpleRag:
#   def __init__(self):
#     self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
#     self.documents = []
#     self.embeddings = None
#     self.tokenizer = AutoTokenizer.from_pretrained("gpt2")
#     self.model = AutoModelForCausalLM.from_pretrained("gpt2")

#   def add_documents(self, documents):
#     self.documents.extend(documents)
#     self.embeddings = self.embedding_model.encode(self.documents)

#   def query(self,question,k=2):
#     question_embedding = self.embedding_model.encode(question)
#     similarities = cosine_similarity([question_embedding],self.embeddings)[0]
#     top_k_indices = np.argsort(similarities)[-k:][::-1]
#     result = []
#     for i in top_k_indices:
#       result.append({'document':self.documents[i], 'score':similarities[i]})
#     return result

#   def generate_text(self,question):
#     retrieved_docs = self.query(question)
#     context = " ".join([doc['document'] for doc in retrieved_docs])
#     prompt = f"""Context: {context}
#     Question: {question}
#     Answer:"""
#     inputs = self.tokenizer(prompt, return_tensors="pt")
#     outputs = self.model.generate(**inputs, max_length=200)
#     answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return answer


In [111]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

class SimpleRag:
    def __init__(self):
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.documents = []
        self.embeddings = None

        self.tokenizer = AutoTokenizer.from_pretrained("gpt2")
        self.model = AutoModelForCausalLM.from_pretrained("gpt2")

    def add_documents(self, documents):
        self.documents.extend(documents)
        self.embeddings = self.embedding_model.encode(self.documents)

    def query(self, question, k=2):
        question_embedding = self.embedding_model.encode(question)
        similarities = cosine_similarity([question_embedding], self.embeddings)[0]
        top_k_indices = np.argsort(similarities)[-k:][::-1]
        result = []
        for i in top_k_indices:
            result.append({'document': self.documents[i], 'score': similarities[i]})
        return result

    def generate_text(self, question):
        retrieved_docs = self.query(question)
        context = "\n".join([doc['document'] for doc in retrieved_docs])
        prompt = f"""
        Basándote en el siguiente contexto, genera una respuesta concisa y clara.

        Contexto:
        {context}

        Pregunta: {question}
        Respuesta:"""

        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=500)
        try:
            outputs = self.model.generate(
                **inputs,
                max_length=500,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                no_repeat_ngram_size=3,
                early_stopping=True
            )

            answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            answer = answer.split("Respuesta:")[-1].strip()

            return answer

        except Exception as e:
            return f"Error generando respuesta: {str(e)}"

def main():
    documents = [
        "iPhone 14 Pro: Smartphone de última generación con pantalla de 6.1 pulgadas, cámara de 48MP y chip A16 Bionic. Precio: $999",
        "Samsung Galaxy S23: Teléfono Android con pantalla AMOLED de 6.1 pulgadas, cámara de 50MP y procesador Snapdragon 8. Precio: $799",
        "MacBook Air M2: Laptop ultradelgada con chip M2, 8GB RAM, 256GB SSD y pantalla Retina de 13.6 pulgadas. Precio: $1199",
        "Sony WH-1000XM5: Auriculares inalámbricos con cancelación de ruido, 30 horas de batería y audio de alta resolución. Precio: $399",
        "Nintendo Switch OLED: Consola de videojuegos con pantalla OLED de 7 pulgadas, 64GB de almacenamiento y modo portátil. Precio: $349",
        "iPad Air 2022: Tablet con chip M1, pantalla de 10.9 pulgadas, compatible con Apple Pencil y Magic Keyboard. Precio: $599",
        "Dell XPS 15: Laptop para profesionales con Intel i7, 16GB RAM, 512GB SSD y pantalla 4K de 15.6 pulgadas. Precio: $1799",
        "AirPods Pro 2: Auriculares TWS con cancelación activa de ruido, audio espacial y resistencia al agua IPX4. Precio: $249",
        "PlayStation 5: Consola de nueva generación con SSD ultrarrápido, ray tracing y control DualSense. Precio: $499",
        "Samsung QN90B: Smart TV QLED 4K de 65 pulgadas con tasa de refresco de 120Hz y HDR. Precio: $1999"
    ]

    print("Inicializando RAG...")
    rag = SimpleRag()

    print("Agregando documentos...")
    rag.add_documents(documents)

    questions = [
        "tipos de laptop"
    ]

    for question in questions:
        print(f"\nPregunta: {question}")

        retrieved_docs = rag.query(question)
        print("\nDocumentos relevantes:")
        for doc in retrieved_docs:
            print(f"Score: {doc['score']:.4f} - {doc['document']}")

        result = rag.generate_text(question)
        print(f"\nRespuesta generada: {result}")

if __name__ == "__main__":
    main()

Inicializando RAG...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Agregando documentos...

Pregunta: tipos de laptop

Documentos relevantes:
Score: 0.4959 - Dell XPS 15: Laptop para profesionales con Intel i7, 16GB RAM, 512GB SSD y pantalla 4K de 15.6 pulgadas. Precio: $1799
Score: 0.4392 - MacBook Air M2: Laptop ultradelgada con chip M2, 8GB RAM, 256GB SSD y pantalla Retina de 13.6 pulgadas. Precio: $1199


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(



Respuesta generada: piedra su puede entre con clara y con una con comunidad.
Pregunta en la forma de novie:
I am not a fan of the way the MacBook Pro is handled by the Intel i5, i7 and i9. I think that the MacBook is the better option because it uses the same processor and RAM as the MacBook.
I believe that iMacs are better for the same reason. I'm not going to go into details about the iMac Pro because I'm sure that I won't get much information from my review.
The iMac is great for the amount of RAM it has. I have a few of my MacBook Pro's on my desk. I like the fact that the Pro features a lot of RAM.
One of the things that I think the MacBook Pros do well is the performance of the graphics cards. I would like to see the graphics card drivers for the i7-6700K.
There are two kinds of cards in the iBook Pro. One is the XPS 14 GPU. The other is the NVIDIA GeForce GTX 1050M. I will try to give the i5 and i7 cards a try as well.
If you have any questions, feel free to ask them on our for